In [5]:
#if(!require("knitr")) install.packages("knitr")
if(!require("tree")) install.packages("tree")
if(!require("readr")) install.packages("readr")
#if(!require("knitr")) install.packages("knitr")
if(!require("caret")) install.packages("caret")
#if(!require("gridExtra")) install.packages("gridExtra")
if(!require("randomForest")) install.packages("randomForest")
if(!require("foreach")) install.packages("foreach")
if(!require("e1071")) install.packages("e1071")
if(!require("furrr")) install.packages("furrr")
if(!require("parallel")) install.packages("parellel")
if(!require("doParallel")) install.packages("doParallel")
if(!require("gbm")) install.packages("gbm")




Loading required package: tree

Registered S3 method overwritten by 'tree':
  method     from
  print.tree cli 

Loading required package: randomForest

randomForest 4.6-14

Type rfNews() to see new features/changes/bug fixes.


Attaching package: ‘randomForest’


The following object is masked from ‘package:dplyr’:

    combine


The following object is masked from ‘package:ggplot2’:

    margin


Loading required package: e1071

Loading required package: furrr

Loading required package: future


Attaching package: ‘future’


The following object is masked from ‘package:caret’:

    cluster


Loading required package: gbm

Loaded gbm 2.1.5



In [0]:
require("readr")
require("tidyverse")
#require("knitr")
require("caret")
require("parallel")
require("doParallel")


In [7]:
# data parsing
soldat <- read_csv("soldat.csv")
soldat$y <- factor(soldat$y,levels = c(-1,1),labels = c("insoluble","soluble"))
n <- nrow(soldat)
p <- ncol(soldat)

Parsed with column specification:
cols(
  .default = col_double()
)

See spec(...) for full column specifications.



In [0]:
cluster <- makeCluster(detectCores() - 1) # convention to leave 1 core for OS
registerDoParallel(cluster)

In [0]:
v <- cor(soldat %>% select(-y), use = "pairwise.complete.obs")
high_corr <- colnames(soldat)[findCorrelation(v, cutoff = 0.95)]
soldat <- soldat %>% select(-all_of(high_corr))

In [0]:
set.seed(1234)
inTest <- createDataPartition(soldat$y,p = 0.5, list = FALSE)[,1]
test <- soldat[inTest,]
training <- soldat[-inTest,]

In [0]:
control <- trainControl(method = "repeatedcv", 
                        number = 5,
                        repeats = 3, 
                        returnResamp = "all",
                        classProbs = TRUE,
                        allowParallel = TRUE)

metric <- "Accuracy"

grid <- expand.grid(.interaction.depth = 1, 
                    .n.trees = c(1, 10, 100, 250, 500, 750, 1000, 1250, 1500, 1750, 2000, 2250, 2500, 2750, 3000, 3250, 3500), 
                    .shrinkage = c(.0001,.001,.01,.1,1),  
                    .n.minobsinnode = 10
                    )

stumps_2000_model <- train(y ~. , 
                           data = training, 
                           method = "gbm", 
                           bag.fraction = 0.5,
                           distribution = "adaboost",
                           trControl = control,
                           tuneGrid = grid,
                           verbose = FALSE,
                           metric = metric)

save(stumps_2000_model,file="stumps_2000_model.Rdata")

In [12]:
print(stumps_2000_model)

Stochastic Gradient Boosting 

2815 samples
  46 predictor
   2 classes: 'insoluble', 'soluble' 

No pre-processing
Resampling: Cross-Validated (5 fold, repeated 3 times) 
Summary of sample sizes: 2253, 2252, 2251, 2252, 2252, 2252, ... 
Resampling results across tuning parameters:

  shrinkage  n.trees  Accuracy   Kappa    
  1e-04         1     0.6202487  0.0000000
  1e-04        10     0.6202487  0.0000000
  1e-04       100     0.6202487  0.0000000
  1e-04       250     0.6202487  0.0000000
  1e-04       500     0.6202487  0.0000000
  1e-04       750     0.6202487  0.0000000
  1e-04      1000     0.6202487  0.0000000
  1e-04      1250     0.6202487  0.0000000
  1e-04      1500     0.6202487  0.0000000
  1e-04      1750     0.6202487  0.0000000
  1e-04      2000     0.6202487  0.0000000
  1e-04      2250     0.6202487  0.0000000
  1e-04      2500     0.6202487  0.0000000
  1e-04      2750     0.6202487  0.0000000
  1e-04      3000     0.6202487  0.0000000
  1e-04      3250     0.6202

In [13]:
# Making the interaction depth vary

control <- trainControl(method = "repeatedcv", 
                        number = 5, 
                        repeats = 3,
                        returnResamp = "all",
                        classProbs = TRUE,
                        allowParallel = TRUE)

metric <- "Accuracy"

grid <- expand.grid(.interaction.depth = c(1,4,8,16), 
                    .n.trees = c(1, 10, 100, 250, 500, 750, 1000, 1250, 1500, 1750, 2000, 2250, 2500, 2750, 3000, 3250, 3500), 
                    .shrinkage = c(.0001,.001,.01,.1,1),  
                    .n.minobsinnode = 10
)

multi_lvl_model <- train(y ~. , 
                         data = training, 
                         method = "gbm", 
                         bag.fraction = 0.5,
                         distribution = "adaboost",
                         trControl = control,
                         tuneGrid = grid,
                         verbose = FALSE,
                         metric = metric)

multi_lvl_model

save(multi_lvl_model,file="multi_lvl_model.Rdata")

Stochastic Gradient Boosting 

2815 samples
  46 predictor
   2 classes: 'insoluble', 'soluble' 

No pre-processing
Resampling: Cross-Validated (5 fold, repeated 3 times) 
Summary of sample sizes: 2252, 2253, 2251, 2252, 2252, 2252, ... 
Resampling results across tuning parameters:

  shrinkage  interaction.depth  n.trees  Accuracy   Kappa     
  1e-04       1                    1     0.6202488  0.00000000
  1e-04       1                   10     0.6202488  0.00000000
  1e-04       1                  100     0.6202488  0.00000000
  1e-04       1                  250     0.6202488  0.00000000
  1e-04       1                  500     0.6202488  0.00000000
  1e-04       1                  750     0.6202488  0.00000000
  1e-04       1                 1000     0.6202488  0.00000000
  1e-04       1                 1250     0.6202488  0.00000000
  1e-04       1                 1500     0.6202488  0.00000000
  1e-04       1                 1750     0.6202488  0.00000000
  1e-04       1        

In [14]:
print(multi_lvl_model)

Stochastic Gradient Boosting 

2815 samples
  46 predictor
   2 classes: 'insoluble', 'soluble' 

No pre-processing
Resampling: Cross-Validated (5 fold, repeated 3 times) 
Summary of sample sizes: 2252, 2253, 2251, 2252, 2252, 2252, ... 
Resampling results across tuning parameters:

  shrinkage  interaction.depth  n.trees  Accuracy   Kappa     
  1e-04       1                    1     0.6202488  0.00000000
  1e-04       1                   10     0.6202488  0.00000000
  1e-04       1                  100     0.6202488  0.00000000
  1e-04       1                  250     0.6202488  0.00000000
  1e-04       1                  500     0.6202488  0.00000000
  1e-04       1                  750     0.6202488  0.00000000
  1e-04       1                 1000     0.6202488  0.00000000
  1e-04       1                 1250     0.6202488  0.00000000
  1e-04       1                 1500     0.6202488  0.00000000
  1e-04       1                 1750     0.6202488  0.00000000
  1e-04       1        